In [1]:
import time
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

#  ------------------------------------------------------------------------ Principal function -----------------------------------------------------------------------------------------
#enter credentials
account_name = 'oficialblobsa'
account_key = 'UpmTirx9Y1tMVUXSZNuEkAw+Rb/MBo7u9R3z8WTV5iO8C+KZXy1+4TA86c1iqIZcSXAI6zX2HqvY+AStaworKQ=='
container_name = 'capexfiles'
container_output_name = 'bronze'

#create a client to interact with blob storage
connect_str = 'DefaultEndpointsProtocol=https;AccountName=' + account_name + ';AccountKey=' + account_key + ';EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

#use the client to connect to the container
container_client = blob_service_client.get_container_client(container_name)
container_client

# Selecione o blob
blob_output_name = "CapexBEForecast.csv"

#downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
#blob_text = downloader.readall()
blob_list = []
final_columns = dict(Account=[], Description=[], Comit=[], variable=[],value=[],Projeto=[],Order=[])
df_final = pd.DataFrame(final_columns)
sas_i = generate_blob_sas(account_name = account_name,
                                container_name = container_name,
                                blob_name = 'AL-043.xlsx',
                                account_key=account_key,
                                permission=BlobSasPermissions(read=True),
                                expiry=datetime.utcnow() + timedelta(hours=1))

sas_url = 'https://' + account_name+'.blob.core.windows.net/' + container_name + '/' + 'AL-043.xlsx'+ '?' + sas_i
df = pd.read_excel(sas_url,sheet_name='Schedule')

            #variables
project_name = df.loc[1, 'Unnamed: 19']
order = df.loc[0, 'Unnamed: 2']
next_year = df.loc[3, 'Unnamed: 16']
current_year = df.loc[89, 'Unnamed: 16']

#Dropping columns)
df = df.drop(labels = 'Unnamed: 0',axis =1)
df = df.drop(labels = 'Unnamed: 4',axis =1)
df = df.drop(labels = 'Unnamed: 5',axis =1)
df = df.drop(labels = 'Unnamed: 6',axis =1)
df = df.drop(labels = 'Unnamed: 7',axis =1)
df = df.drop(labels = 'Unnamed: 8',axis =1)
df = df.drop(labels = 'Unnamed: 9',axis =1)
df = df.drop(labels = 'Unnamed: 10',axis =1)
df = df.drop(labels = 'Unnamed: 11',axis =1)
df = df.drop(labels = 'Unnamed: 12',axis =1)
df = df.drop(labels = 'Unnamed: 13',axis =1)
df = df.drop(labels = 'Unnamed: 14',axis =1)
df = df.drop(labels = 'Unnamed: 15',axis =1)
df = df.drop(labels = 'Unnamed: 17',axis =1)
df = df.drop(labels = 'Unnamed: 18',axis =1)
df = df.drop(labels = 'Unnamed: 31',axis =1)
df = df.drop(labels = 'Unnamed: 32',axis =1)
df = df.drop(labels = 'Unnamed: 33',axis =1)
df = df.drop(labels = 'Unnamed: 34',axis =1)
#Dropping Lines
df = df.drop(labels = [0,1,4,5,6],axis = 0)
df.reset_index(drop=True)
maxindex = df.index.max()
for i in range(172,maxindex+1):
    df = df.drop(labels =[i],axis = 0)
#adicionando mês como cabeçalho
df = df.rename(columns=df.iloc[1]).drop(df.index[1])
df.rename(columns={np.nan: 'Comit'}, inplace=True)
df.rename(columns={'variable    ': 'Month'}, inplace=True)
#Adicionando colune de ano baseado no INdex e variaveis previamente criadas
df['Ano'] = df.index.to_series().apply(lambda x: next_year if x <=86 else current_year)
#preenchendo valores nulos
df['Account'].fillna(method='ffill', inplace=True)
df['Description'].fillna(method='ffill', inplace=True)
#Exluding total column and dropping lines
df = df.drop(df.columns[3], axis = 1)
df = df.drop(labels = [87,88,89,90,92],axis = 0)
#unpivot
df = df.melt(id_vars=['Account','Description','Comit','Ano'], value_vars=['Jan', 'Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
#creating columns
df['Projeto'] = project_name
df['Order'] = order
#Filtering tb comit
df = df[df["Comit"] == "tb"]
df = df[df["Account"] != "Total"]
df = df[df["Account"] != "1"]
df = df[df["Account"] != "2"]
df = df[df["Account"] != "3"]
df = df[df["Account"] != "4"]
df = df[df["Account"] != "5"]
df = df[df["Account"] != "6"]
df = df[df["Account"] != "7"]
df = df[df["Account"] != "8"]
df = df[df["Account"] != "9"]
df = df[df["Account"] != "10"]

#Reset Index
df.reset_index(inplace=True)

time.sleep(0.5)

df_final = pd.concat([df, df_final])
df_final['variable'].replace({'Jan': '01', 
            'Feb': '02',
            'Mar': '03',
            'Apr': '04',
            'May': '05',
            'Jun': '06',
            'Jul': '07',
            'Aug': '08',
            'Sep': '09',
            'Oct': '10',
            'Nov': '11',
            'Dec': '12'
            }, inplace = True)

 

C:\Users\wgonzaga\AppData\Local\Temp\ipykernel_24384\2168064465.py:79: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Account'].fillna(method='ffill', inplace=True)
C:\Users\wgonzaga\AppData\Local\Temp\ipykernel_24384\2168064465.py:80: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Description'].fillna(method='ffill', inplace=True)


In [9]:
df.to_csv(r'C:\Users\wgonzaga\Downloads\dfteste.csv')

In [9]:
df_final.dtypes

index          float64
Account         object
Description     object
Comit           object
Ano            float64
variable        object
value           object
Projeto         object
Order           object
dtype: object

In [13]:
df_final[df_final["Account"] != 1]

,index,Account,Description,Comit,Ano,variable,value,Projeto,Order
0,2.0,660011,Capital Purchases,tb,2025.0,01,0,AL-043 SCR Boiler H Panel Replacement and Mech...,B4170020003F
1,4.0,660012,Equipment,tb,2025.0,01,0,AL-043 SCR Boiler H Panel Replacement and Mech...,B4170020003F
3,8.0,2,Alfalaval,tb,2025.0,01,0,AL-043 SCR Boiler H Panel Replacement and Mech...,B4170020003F
4,10.0,3,Refratômetros,tb,2025.0,01,0,AL-043 SCR Boiler H Panel Replacement and Mech...,B4170020003F
5,12.0,4,Inversor de frequencia,tb,2025.0,01,0,AL-043 SCR Boiler H Panel Replacement and Mech...,B4170020003F
...,...,...,...,...,...,...,...,...,...
907,1923.0,660023,Outside Engineering,tb,2024.0,12,3896.356906,AL-043 SCR Boiler H Panel Replacement and Mech...,B4170020003F
908,1925.0,660024,Miscellaneous,tb,2024.0,12,0,AL-043 SCR Boiler H Panel Replacement and Mech...,B4170020003F
909,1927.0,660025,Indirect Costs (Internal Engineering),tb,2024.0,12,3896.356906,AL-043 SCR Boiler H Panel Replacement and Mech...,B4170020003F
910,1929.0,660026,Other Assets,tb,2024.0,12,0,AL-043 SCR Boiler H Panel Replacement and Mech...,B4170020003F
